In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
from scipy import stats
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import os
num_frames = 30
actions = ["hello","thanks","call"]

In [2]:
ACCURACY_THRESHOLD = 0.90

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))
            self.model.stop_training = True

# Instantiate a callback object
accuracy_callback = myCallback()

In [3]:
actions = np.asarray(actions)
# specify folder where you will store action data
DATA_PATH = os.path.join('GestureDetectionData')

label_map = {label:num for num, label in enumerate(actions)}
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(num_frames):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

log_dir = os.path.join('logs-1')
tb_callback = TensorBoard(log_dir=log_dir)
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy','accuracy'])
# model.fit(X_train, y_train, epochs=epochs, callbacks=[tb_callback])
# print(model.summary())
# model.save('sign-language-detection_model-weights.h5')

Metal device set to: Apple M1


2022-12-12 00:37:14.222175: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-12 00:37:14.222758: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback,accuracy_callback])

Epoch 1/1000


2022-12-12 00:37:15.797757: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-12 00:37:16.501911: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 36s 3s/step - loss: 9.0805 - categorical_accuracy: 0.3265 - accuracy: 0.3265
Epoch 2/1000
11/11 [==============================] - 29s 3s/step - loss: 7.7762 - categorical_accuracy: 0.3412 - accuracy: 0.3412
Epoch 3/1000
11/11 [==============================] - 34s 3s/step - loss: 13.5009 - categorical_accuracy: 0.3647 - accuracy: 0.3647
Epoch 4/1000
11/11 [==============================] - 33s 3s/step - loss: 23.2682 - categorical_accuracy: 0.3471 - accuracy: 0.3471
Epoch 5/1000
11/11 [==============================] - 35s 3s/step - loss: 24.1589 - categorical_accuracy: 0.2765 - accuracy: 0.2765
Epoch 6/1000
11/11 [==============================] - 35s 3s/step - loss: 153.7738 - categorical_accuracy: 0.3382 - accuracy: 0.3382
Epoch 7/1000
11/11 [==============================] - 36s 3s/step - loss: 37.4176 - categorical_accuracy: 0.3706 - accuracy: 0.3706
Epoch 8/1000
11/11 [==============================] - 35s 3s/step - loss: 68.5457 - catego

KeyboardInterrupt: 

In [39]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 30, 64)            442112    
                                                                 
 lstm_19 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_20 (LSTM)              (None, 64)                49408     
                                                                 
 dense_18 (Dense)            (None, 64)                4160      
                                                                 
 dense_19 (Dense)            (None, 32)                2080      
                                                                 
 dense_20 (Dense)            (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

In [41]:
res = model.predict(X_test)

2022-12-11 23:39:40.231778: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 850ms/step


In [44]:
actions[np.argmax(res[1])]

'hello'

In [45]:
actions[np.argmax(y_test[1])]

'hello'

In [40]:
model.save('lstm_weights_gesturerecognition.h5')

In [51]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


model.load_weights("lstm_weights_gesturerecognition.h5")

yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)
print(accuracy_score(ytrue, yhat))

2022-12-11 23:40:22.132325: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 381ms/step
0.8333333333333334
